In [90]:
import MetaTrader5 as mt5
import pandas as pd
import pytz
from datetime import datetime
from time import gmtime, strftime

def initialize():

    login = 52029003
    password = '0a6Dx9o5!HnsaC'
    server = 'ICMarketsEU-Demo'
    
    # get current date and time
    if not mt5.initialize("C:/Program Files/FTMO MetaTrader 5 - Copy/terminal64.exe"):
        print("initialize() failed, error code =", mt5.last_error())
        quit()
    else:
        print("initialize() success")
        tz_kyjev = pytz.timezone('Europe/Kiev')
        datetime_kyjev = datetime.now(tz_kyjev)
        print("Time: ",datetime_kyjev.strftime("%H:%M:%S"))

    mt5.login(login, password, server)

    acc_info = mt5.account_info()
    if acc_info is None:
        print("Failed to get account info")

    login_number = acc_info.login
    print("Login number: ", login_number)
        # establish connection to the MetaTrader 5 terminal

initialize()

bars = mt5.copy_rates_from_pos("UK100", mt5.TIMEFRAME_M5, 0, 99999)
df = pd.DataFrame(bars)  
df

initialize() success
Time:  22:36:38
Login number:  52029003


,time,open,high,low,close,tick_volume,spread,real_volume
0,1689617400,7415.0,7415.5,7410.7,7410.7,79,100,0
1,1689617700,7410.7,7413.5,7410.2,7412.7,106,100,0
2,1689618000,7413.2,7416.2,7412.7,7416.2,81,100,0
3,1689618300,7415.7,7415.7,7411.2,7412.2,75,100,0
4,1689618600,7412.2,7412.7,7403.7,7405.7,161,200,0
...,...,...,...,...,...,...,...,...
99994,1734992100,8111.0,8111.8,8105.0,8106.0,107,100,0
99995,1734992400,8106.3,8111.5,8106.3,8111.5,80,100,0
99996,1734992700,8111.8,8114.8,8111.8,8112.8,72,100,0
99997,1734993000,8113.0,8114.3,8111.5,8114.0,110,100,0


In [91]:
def split_by_next_row_three_ways(df):
    """
    Split DataFrame into three based on next row's open/close relationship
    
    Parameters:
        df (pandas.DataFrame): DataFrame with 'open' and 'close' columns
    
    Returns:
        tuple: (price_increased, price_equal, price_decreased)
    """
    # Create conditions for each case
    increased_condition = df['open'] < df['close']
    equal_condition = df['open'] == df['close']
    
    # Shift conditions UP by 1 (looking at next row's conditions)
    next_increased = increased_condition.shift(-1)
    next_equal = equal_condition.shift(-1)
    
    # Fill NaN values with False
    next_increased = next_increased.fillna(False)
    next_equal = next_equal.fillna(False)
    
    # Creating the three separate DataFrames
    price_increased = df[next_increased]
    price_equal = df[next_equal]
    price_decreased = df[~(next_increased | next_equal)]  # Everything else
    
    return price_increased, price_equal, price_decreased

increased, equal, decreased = split_by_next_row_three_ways(df)
increased = increased.copy()
equal = equal.copy()
decreased = decreased.copy()    

print("Rows where next row has open < close:")
print(increased)
print("\nRows where next row has open == close:")
print(equal)
print("\nRows where next row has open > close:")
print(decreased)

Rows where next row has open < close:
             time    open    high     low   close  tick_volume  spread  \
0      1689617400  7415.0  7415.5  7410.7  7410.7           79     100   
1      1689617700  7410.7  7413.5  7410.2  7412.7          106     100   
4      1689618600  7412.2  7412.7  7403.7  7405.7          161     200   
5      1689618900  7406.2  7409.7  7405.2  7409.7          138     200   
8      1689619800  7410.2  7410.7  7409.7  7410.2           33     200   
...           ...     ...     ...     ...     ...          ...     ...   
99990  1734990900  8111.8  8113.0  8111.0  8111.3           52     100   
99994  1734992100  8111.0  8111.8  8105.0  8106.0          107     100   
99995  1734992400  8106.3  8111.5  8106.3  8111.5           80     100   
99996  1734992700  8111.8  8114.8  8111.8  8112.8           72     100   
99997  1734993000  8113.0  8114.3  8111.5  8114.0          110     100   

       real_volume  
0                0  
1                0  
4         

In [92]:
#Increased => increased bar before, checking short

#Get values for expected direction 
increased['open_low_difference'] = increased['open'] - increased['low']
increased['sl'] = increased['high'] - increased['open']

decreased['high_open_difference'] = decreased['high'] - decreased['open']
decreased['sl'] = decreased['open'] - decreased['low']
decreased

,time,open,high,low,close,tick_volume,spread,real_volume,high_open_difference,sl
2,1689618000,7413.2,7416.2,7412.7,7416.2,81,100,0,3.0,0.5
3,1689618300,7415.7,7415.7,7411.2,7412.2,75,100,0,0.0,4.5
12,1689621000,7413.0,7416.7,7413.0,7416.7,18,200,0,3.7,0.0
15,1689621900,7416.0,7417.0,7416.0,7416.7,32,200,0,1.0,0.0
18,1689622800,7413.7,7416.7,7413.7,7416.2,28,200,0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...
99989,1734990600,8113.8,8115.3,8110.0,8112.0,137,100,0,1.5,3.8
99991,1734991200,8111.0,8112.5,8109.5,8112.5,84,100,0,1.5,1.5
99992,1734991500,8112.3,8115.3,8111.3,8111.5,84,100,0,3.0,1.0
99993,1734991800,8111.8,8112.0,8108.5,8110.5,80,100,0,0.2,3.3


In [93]:
shortTP = increased['open_low_difference'].mean()
shortSL = increased['sl'].mean()
print(shortTP,shortSL)

2.2253215134034585 2.144748260943334


In [94]:
longTP = decreased['high_open_difference'].mean()
longSL = decreased['sl'].mean()
print(longTP,longSL)


2.206235375983994 2.1653079198016765


In [95]:
increased[:99999]

,time,open,high,low,close,tick_volume,spread,real_volume,open_low_difference,sl
0,1689617400,7415.0,7415.5,7410.7,7410.7,79,100,0,4.3,0.5
1,1689617700,7410.7,7413.5,7410.2,7412.7,106,100,0,0.5,2.8
4,1689618600,7412.2,7412.7,7403.7,7405.7,161,200,0,8.5,0.5
5,1689618900,7406.2,7409.7,7405.2,7409.7,138,200,0,1.0,3.5
8,1689619800,7410.2,7410.7,7409.7,7410.2,33,200,0,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...
99990,1734990900,8111.8,8113.0,8111.0,8111.3,52,100,0,0.8,1.2
99994,1734992100,8111.0,8111.8,8105.0,8106.0,107,100,0,6.0,0.8
99995,1734992400,8106.3,8111.5,8106.3,8111.5,80,100,0,0.0,5.2
99996,1734992700,8111.8,8114.8,8111.8,8112.8,72,100,0,0.0,3.0


In [96]:
increased[increased.sl <= increased['sl'].mean()]

,time,open,high,low,close,tick_volume,spread,real_volume,open_low_difference,sl
0,1689617400,7415.0,7415.5,7410.7,7410.7,79,100,0,4.3,0.5
4,1689618600,7412.2,7412.7,7403.7,7405.7,161,200,0,8.5,0.5
8,1689619800,7410.2,7410.7,7409.7,7410.2,33,200,0,0.5,0.5
10,1689620400,7411.2,7411.2,7410.7,7411.2,21,200,0,0.5,0.0
11,1689620700,7411.2,7412.7,7411.2,7412.7,6,200,0,0.0,1.5
...,...,...,...,...,...,...,...,...,...,...
99984,1734989100,8119.0,8119.8,8117.8,8119.0,109,100,0,1.2,0.8
99986,1734989700,8121.3,8121.5,8117.5,8117.5,142,100,0,3.8,0.2
99990,1734990900,8111.8,8113.0,8111.0,8111.3,52,100,0,0.8,1.2
99994,1734992100,8111.0,8111.8,8105.0,8106.0,107,100,0,6.0,0.8


In [97]:
#Difference in %, all incresed bars vs bars where O - L is bigger than average 
rows, columns =increased[increased.open_low_difference >= increased['open_low_difference'].mean()].shape
rw, col = increased.shape
print((rows/rw)*100,'%')

34.751442144553785 %


In [98]:
#Difference in %, all incresed bars vs bars where O - L is 2  or more
rows, columns =increased[increased.open_low_difference >= 2].shape
rw, col = increased.shape
print((rows/rw)*100,'%')

41.63980318968443 %


In [99]:
rows, columns = increased[increased.sl >= increased.open_low_difference.mean()].shape
print((rows/rw)*100,'%')


34.1364099083814 %


In [100]:
a = increased[increased.sl <= increased['sl'].mean()]
a.open_low_difference.mean()
#a = a[a.open_low_difference >= a.open_low_difference.mean()] 


2.584037108544124

In [101]:
a = increased[increased.sl <= increased['sl'].mean()]

a = a[a.open_low_difference >= a.open_low_difference.mean()] 
a.open_low_difference.mean()

5.324912017970793